# Basic Search Example

This notebook demonstrates the basic usage of the nanohub-results library to search and retrieve simulation data from nanoHUB.

## What you'll learn:
- How to authenticate with the nanoHUB API
- How to explore tool schemas
- How to build and execute basic queries
- How to access result data

## Setup and Authentication

First, we need to set up authentication with the nanoHUB API. You'll need a personal access token which you can generate at https://nanohub.org/developer.

**Note:** Never commit your actual token to version control. Consider using environment variables or a config file that's in `.gitignore`.

In [ ]:
# Add parent directory to path for local development
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

In [ ]:
import nanohubremote as nr
from nanohubresults import Results

# Initialize session with authentication
# Replace with your actual token from https://nanohub.org/developer
auth_data = {
    "grant_type": "personal_token",
    "token": "YOUR_TOKEN_HERE"
}
session = nr.Session(auth_data, url="https://nanohub.org/api")
results = Results(session)

print("✓ Successfully authenticated with nanoHUB API")

## Exploring Tool Schemas

Before querying data, it's useful to understand what fields are available for a given tool. We'll use the `get_tool_detail()` method to fetch the schema for the 2dfets tool (2D Field Effect Transistor Simulator).

In [ ]:
# Get the schema for 2dfets to see what fields are available
print("Fetching tool schema...")
schema = results.get_tool_detail("2dfets", simtool=False)

# Extract input and output fields
tool_data = schema['results'][0]['2dfets']
input_fields = list(tool_data['input'].keys())
output_fields = list(tool_data['output'].keys())

print(f"\nAvailable input fields ({len(input_fields)} total):")
print(f"  First 5: {input_fields[:5]}")
print(f"\nAvailable output fields ({len(output_fields)} total):")
print(f"  First 5: {output_fields[:5]}")

## Building and Executing a Query

The nanohub-results library provides a fluent query builder interface. You can:
- Filter results based on input/output field values
- Select specific fields to return
- Limit the number of results
- Sort results

Let's search for results where the Fermi energy (`input.Ef`) is greater than 0.

In [ ]:
# Build a query
# Note: 2dfets is a Rappture tool, so simtool=False (default)
# For Sim2L tools like alphafold231, use simtool=True
query = results.query("2dfets", simtool=False) \
    .filter("input.Ef", ">", 0) \
    .select("input.Ef", "output.f41") \
    .limit(10)

# Execute the query
print("Executing query...")
response = query.execute()

print(f"\n✓ Query completed successfully")
print(f"  Results found: {len(response.get('results', []))}")
print(f"  Search time: {response.get('searchTime', 0):.4f} seconds")

## Accessing Result Data

Each result includes:
- A unique identifier (SQUID)
- The requested input and output field values
- Metadata like revision number and date

In [ ]:
# Display the first result
if response.get('results') and len(response['results']) > 0:
    first_result = response['results'][0]
    
    print("First result details:")
    print(f"  SQUID: {first_result.get('squid')}")
    print(f"  Fermi Energy: {first_result.get('input.Ef')} V")
    
    # The output.f41 field contains curve data
    output_data = first_result.get('output.f41', {})
    if output_data:
        print(f"\n  Output curve (f41):")
        print(f"    X-axis points: {len(output_data.get('xaxis', []))}")
        print(f"    Y-axis points: {len(output_data.get('yaxis', []))}")
        print(f"    Sample X values: {output_data.get('xaxis', [])[:3]}")
        print(f"    Sample Y values: {output_data.get('yaxis', [])[:3]}")
else:
    print("No results found.")

## Visualizing Results (Optional)

If you have matplotlib installed, you can visualize the output curves.

In [ ]:
# Uncomment if you have matplotlib installed
# import matplotlib.pyplot as plt

# if response.get('results') and len(response['results']) > 0:
#     first_result = response['results'][0]
#     output_data = first_result.get('output.f41', {})
#     
#     if output_data:
#         plt.figure(figsize=(10, 6))
#         plt.plot(output_data['xaxis'], output_data['yaxis'], 'b-o')
#         plt.xlabel('Voltage (V)')
#         plt.ylabel('Energy (eV)')
#         plt.title(f'2DFETS Output Curve - SQUID: {first_result.get("squid")}')
#         plt.grid(True, alpha=0.3)
#         plt.show()

## Summary

In this notebook, you learned:
1. ✓ How to authenticate with the nanoHUB API
2. ✓ How to explore available fields using `get_tool_detail()`
3. ✓ How to build queries using the fluent interface
4. ✓ How to execute queries and access results

## Next Steps

Check out the other example notebooks:
- `02_advanced_filtering.ipynb` - Multiple filters and sorting
- `03_downloading_files.ipynb` - Accessing and saving result data
- `04_pagination.ipynb` - Iterating over large result sets